In [1]:
import numpy as np
import pandas as pd

* **Sajeev Yadav**
* **102016073**
* **3CS11**

## Entropy Function

In [2]:
def find_entropy(df):
    Class=df.keys()[0]
    values=df[Class].unique()
    entropy=0
    for value in values:
        prob=len(df[df[Class]==value])/len(df[Class])
        entropy+=-prob*np.log2(prob)
    return entropy

In [3]:
colnames=['Predict' , 'first_val' , 'second_val ', 'third_val' , 'fourth_val']
df=pd.read_csv('/kaggle/input/balance-scale/balance-scale.data',names=colnames, header=None)
print(df)

    Predict  first_val  second_val   third_val  fourth_val
0         B          1            1          1           1
1         R          1            1          1           2
2         R          1            1          1           3
3         R          1            1          1           4
4         R          1            1          1           5
..      ...        ...          ...        ...         ...
620       L          5            5          5           1
621       L          5            5          5           2
622       L          5            5          5           3
623       L          5            5          5           4
624       B          5            5          5           5

[625 rows x 5 columns]


In [4]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df, test_size = 0.2, random_state = 1)

In [5]:
print(train)

    Predict  first_val  second_val   third_val  fourth_val
236       L          2            5          3           2
463       L          4            4          3           4
620       L          5            5          5           1
3         R          1            1          1           4
545       L          5            2          5           1
..      ...        ...          ...        ...         ...
129       R          2            1          1           5
144       R          2            1          4           5
72        R          1            3          5           3
235       L          2            5          3           1
37        R          1            2          3           3

[500 rows x 5 columns]


In [6]:
print(test)

    Predict  first_val  second_val   third_val  fourth_val
552       L          5            3          1           3
23        R          1            1          5           4
41        R          1            2          4           2
598       B          5            4          5           4
370       L          3            5          5           1
..      ...        ...          ...        ...         ...
482       L          4            5          2           3
475       L          4            5          1           1
80        L          1            4          2           1
160       L          2            2          3           1
392       R          4            1          4           3

[125 rows x 5 columns]


In [7]:
print(train.shape)
print(test.shape)

(500, 5)
(125, 5)


In [8]:
find_entropy(train)

1.3119256793719374

## Feature Entropy

In [9]:
def find_information_entropy(df,attribute):
    Class=df.keys()[0]
    labels=df[Class].unique()
    attr_values=df[attribute].unique()
    avg_entropy=0
    for value1 in attr_values:
        entropy_subsample=0
        for value2 in labels:
            num=len(df[attribute][df[attribute]==value1][df[Class]==value2])
            den=len(df[attribute][df[attribute]==value1])
            prob=num/den
            entropy_subsample+=-(prob*np.log2(prob+1e-7))
        weight=den/len(df)
        avg_entropy+=weight*entropy_subsample
    return avg_entropy

In [10]:
print(find_information_entropy(train,'first_val'))
print(find_information_entropy(train,'second_val '))
print(find_information_entropy(train,'third_val'))
print(find_information_entropy(train,'fourth_val'))

1.1692288122098011
1.1733858868305576
1.1744922345235367
1.176082018041286


In [11]:
print(df.keys()[1:])
def find_winner(df):
    IG=[]
    for attribute in df.keys()[1:]:
        IG.append(find_entropy(df)-find_information_entropy(df,attribute))
    print(IG)
    return df.keys()[1:][np.argmax(IG)]

Index(['first_val', 'second_val ', 'third_val', 'fourth_val'], dtype='object')


In [12]:
# find_winner(df)
find_winner(train)

[0.14269686716213625, 0.13853979254137982, 0.1374334448484007, 0.1358436613306513]


'first_val'

In [13]:
def training(df,tree=None):
    Class=df.keys()[0]
    split_attr=find_winner(df)
    split_attr_values=df[split_attr].unique()
    if tree is None:
        tree={}
        tree[split_attr]={}
    for value in split_attr_values:
        sub_df=df[df[split_attr]==value].reset_index(drop=True)
        #Clvalue,count=np.unique(sub_df[Class],return_counts=True)
        Clvalue=sub_df[Class].unique()
        if len(Clvalue)==1:
            tree[split_attr][value]=Clvalue[0]
        else:
            tree[split_attr][value]=training(sub_df)
    return tree

In [14]:
# tree=training(df)
tree=training(train)

[0.14269686716213625, 0.13853979254137982, 0.1374334448484007, 0.1358436613306513]
[4.328084013405231e-07, 0.16062162492187304, 0.2629583235069235, 0.23458434244404747]
[4.3280831629743943e-07, 0.33773221882109994, 4.3280831629743943e-07, 0.5972551769542737]
[4.328084399762844e-07, 1.5219282391568592, 4.328084399762844e-07, 4.328084399762844e-07]
[2.8853896960612246e-07, 0.8112782687286297, 2.8853896960612246e-07, 2.8853896960612246e-07]
[4.3280824435498744e-07, 0.28103631292771636, 4.3280824435498744e-07, 0.2607847666871141]
[2.8853896960612246e-07, 2.8853896960612246e-07, 2.8853896960612246e-07, 0.8112782687286297]
[2.885389757123491e-07, 2.885389757123491e-07, 2.885389757123491e-07, 0.9182959783239865]
[4.328083780258396e-07, 0.4715459542215904, 4.328083780258396e-07, 0.29763292956109033]
[4.328084279858757e-07, 4.328084279858757e-07, 4.328084279858757e-07, 1.3709507387241657]
[4.328084279858757e-07, 4.328084279858757e-07, 4.328084279858757e-07, 1.3709507387241657]
[4.32808441752641

In [15]:
import pprint
# using pretty print
pprint.pprint(tree)

{'first_val': {1: {'third_val': {1: {'fourth_val': {1: 'L',
                                                    2: {'second_val ': {1: 'R',
                                                                        2: 'B',
                                                                        3: 'L',
                                                                        4: 'L'}},
                                                    3: {'second_val ': {1: 'R',
                                                                        2: 'R',
                                                                        3: 'B',
                                                                        4: 'L',
                                                                        5: 'L'}},
                                                    4: {'second_val ': {1: 'R',
                                                                        2: 'R',
                                                        

In [16]:
def predict(inst,tree):
    for nodes in tree.keys():
        value=inst[nodes]
        tree=tree[nodes][value]
        prediction=0
        if type(tree) is dict:
            prediction=predict(inst,tree)
        else:
            prediction=tree
            break;
    return prediction

In [17]:
Y_label=[]
i=1
for i in range(len(train)):
    inst=train.iloc[i,:]
    prediction=predict(inst,tree)
    Y_label.append(prediction)
    

In [18]:
Y_label

['L',
 'L',
 'L',
 'R',
 'L',
 'R',
 'L',
 'L',
 'B',
 'B',
 'L',
 'L',
 'L',
 'R',
 'R',
 'R',
 'R',
 'R',
 'L',
 'L',
 'R',
 'L',
 'L',
 'L',
 'R',
 'R',
 'R',
 'L',
 'R',
 'L',
 'R',
 'L',
 'L',
 'R',
 'L',
 'L',
 'R',
 'R',
 'R',
 'L',
 'L',
 'L',
 'L',
 'R',
 'R',
 'L',
 'R',
 'R',
 'R',
 'L',
 'L',
 'R',
 'L',
 'L',
 'R',
 'L',
 'B',
 'R',
 'R',
 'R',
 'L',
 'L',
 'L',
 'R',
 'L',
 'L',
 'R',
 'L',
 'B',
 'L',
 'R',
 'L',
 'L',
 'R',
 'L',
 'R',
 'B',
 'R',
 'L',
 'B',
 'R',
 'B',
 'L',
 'R',
 'L',
 'L',
 'R',
 'B',
 'L',
 'L',
 'L',
 'B',
 'L',
 'R',
 'R',
 'R',
 'R',
 'L',
 'R',
 'R',
 'L',
 'R',
 'L',
 'L',
 'B',
 'L',
 'L',
 'B',
 'R',
 'L',
 'R',
 'R',
 'L',
 'R',
 'R',
 'L',
 'B',
 'L',
 'L',
 'L',
 'L',
 'R',
 'L',
 'R',
 'L',
 'L',
 'R',
 'L',
 'R',
 'L',
 'R',
 'L',
 'L',
 'L',
 'R',
 'L',
 'B',
 'L',
 'R',
 'R',
 'R',
 'B',
 'R',
 'R',
 'L',
 'R',
 'R',
 'L',
 'R',
 'R',
 'R',
 'L',
 'R',
 'R',
 'R',
 'R',
 'L',
 'L',
 'L',
 'R',
 'L',
 'L',
 'R',
 'L',
 'L',
 'R',
 'R'

In [19]:
# from sklearn import metrics
# print(metrics.classification_report(train.iloc[:,],Y_label))
df_copy=train.iloc[:,0]
df_copy.shape
df_copy

236    L
463    L
620    L
3      R
545    L
      ..
129    R
144    R
72     R
235    L
37     R
Name: Predict, Length: 500, dtype: object

## Accuracy

In [20]:
cnt=0
for i in range(len(df_copy)):
  if(Y_label[i] == df.iloc[i,0]):
    cnt=cnt+1

print("Accuracy = ",cnt/len(df_copy))

Accuracy =  0.412
